<a href="https://colab.research.google.com/github/merovivant/AssistenteVestibular/blob/main/Notebooks/embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Embeddings
Criando os embeddings de texto e armazenando os vetores criados na nuvem do Pinecone

## Importações

In [4]:
!pip -q install openai langchain pinecone-client tiktoken
import os
import getpass
import pinecone
from google.colab import drive
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.text_splitter import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter

drive.mount('/drive')
file_path = os.path.join('/drive/MyDrive/Chatbot/texto.md')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


## Transformações de Texto

In [6]:
#Abrindo o arquivo markdown do drive e transformando em string
with open(file_path, "r") as f:
  texto = f.read()

In [7]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
    ("####", "Header 4"),
]

#Dividindo o texto no nível das tags markdown
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
md_header_splits = markdown_splitter.split_text(texto)

#Dividindo o texto no nível de caracteres
chunk_size = 400
chunk_overlap = 100
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

# Split
splits = text_splitter.split_documents(md_header_splits)

## Código

In [9]:
# Importando as chaves de API necessárias
os.environ["PINECONE_API_KEY"] = getpass.getpass("Pinecone API Key:")
os.environ["PINECONE_ENV"] = getpass.getpass("Pinecone Environment:")
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

Pinecone API Key:··········
Pinecone Environment:··········
OpenAI API Key:··········


In [10]:
embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))

In [14]:
#Inicializando o Pinecone
pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),
    environment=os.getenv("PINECONE_ENV"),
)

index_name = "unicampresolucao"

docsearch = Pinecone.from_documents(splits, embeddings, index_name=index_name)